In [0]:
import pandas as pd
import pymssql
import pyspark.sql.functions as F
import hashlib

In [0]:
jdbcHostname = "54.245.193.229" #"54.245.193.229" 
jdbcDatabase = "DEV" #"KAGGLE" 
jdbcPort = "1433" #1433 
jdbcUsername = "sa"
jdbcPassword = "WindowsSQL-123"

In [0]:
Source_Table_Name = "EMP_INFO"

In [0]:
def get_connection(jdbcHostname, jdbcUsername, jdbcPassword, jdbcDatabase):
    try:
        sql_connection = pymssql.connect(host=jdbcHostname,user=jdbcUsername,password=jdbcPassword,database=jdbcDatabase, as_dict=True, autocommit=True)
#This exception is raised for things that are not in control of the programmer. For e.g unexpected disconnect, error in memory allocation etc, selected database not exists.
    except pymssql.OperationalError as e:
        print("connection failed")
    except:
        print("Unknown error occurred")
    else:
        return sql_connection

In [0]:
def display_data(sql_disqry, sql_conn):
    try:
        cursor = sql_conn.cursor()
        cursor.execute(sql_disqry)
        sql_exec = cursor.fetchone()
        if len(sql_exec) > 0:
            sql_data = pd.DataFrame(cursor.fetchall())
            result = sql_data.head(n=None)
            display(result)
      #create table in data bricks
            spark_df = spark.createDataFrame(result)
            df = spark_df.toPandas()
            #spark_df.orderBy("Emp_Id").write.mode("overwrite").saveAsTable(Source_Table_Name)
            
            pds = spark.sql("(select * FROM {})".format(Source_Table_Name))
            pdf = pds.withColumn("Hash_Key",F.sha2(F.concat('Emp_Id','Emp_name', 'Designation', 'Department', 'City', 'Salary', F.unix_timestamp('Joining_Date')),256))
            #pdf = pds.withColumn("Hash_Key",F.md5(F.concat('Emp_Id', F.trim(F.lower('Emp_name')), F.trim(F.lower('Designation')), F.trim(F.lower('Department')), F.trim(F.lower('City')), 'Salary', F.unix_timestamp('Joining_Date'))))
            pnd = pdf.toPandas()
            pd_df = pnd[['Emp_Id', 'Hash_Key']]
            #df_all = df.compare(pd_df)
            #display(df_all)
        else:
            print("No Record")
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
sql_conn = get_connection(jdbcHostname, jdbcUsername, jdbcPassword, jdbcDatabase)

In [0]:
#sql_disqry ="(select Emp_Id, HASHBYTES('MD5', CONCAT(Emp_Id, '|', trim(lower(Emp_name)), '|', trim(lower(Designation)), '|', trim(lower(Department)),'|', trim(lower(City)), '|', Salary, '|', DATEDIFF(s, '1970-01-01 00:00:00', Joining_Date)))as Hash_Key FROM {})".format(Source_Table_Name)
#sql_disqry ="(select Emp_Id, HASHBYTES('Sha2_256', CONCAT(Emp_Id, Emp_name, Designation, Department, City, Salary))as Hash_Key FROM {})".format(Source_Table_Name)
sql_disqry ="(SELECT Emp_Id, lower(CONVERT(NVARCHAR(256), HASHBYTES('Sha2_256', CONCAT(Emp_Id, Emp_name, Designation, Department, City, Salary,DATEDIFF(s, '1970-01-01 00:00:00', Joining_Date))),2)) as Hash_Key FROM {})".format(Source_Table_Name)
#sql_disqry ="(select * from {})".format(Source_Table_Name)
display_data(sql_disqry, sql_conn)

Emp_Id,Hash_Key
4,dfe448dc6fb5bdb1aeff670d6d0bbd564418613beb725841e803eb0a5ecb2833
6,440a2ad1a4472c9703dfeac22b8a6465004710701b5ece5b25b53aa8f8db7493
7,e1999a3a4b0815464685bdcf1e80bd9635ecf8e1394b3d8d2f692cd4a9024234
8,b5cd98edfd177e0cbf476728375c679147ba61a95cd62a89e919d7cb309525c1
9,bebd1156519378d61fced177c271841cfd30d3ef4af624a81c3e37c9344f42ef
11,12b200ab4af6e9c866382a8e0f0d0770c76e06704030b1feaaa0090e5598a165
12,b6525fd429191431da1120e2e89ab4c426915fdeefacc28d0fc83472a96b7d22
13,4f16df96196a8a2eb75c579ef792b961c2c000e47070264c157a3cea7bcf86ed
14,4367e01f192da2f38fb5612007ccdefacb5477b010077542a396fc4bb0432ba8
15,aaa762d2fbeb1d33dbe7f9bee3b9ac1d809d7b323fe94a73946fc09f54277964


In [0]:
def display_datas(sql_disqry, sql_conn):
    try:
        cursor = sql_conn.cursor()
        cursor.execute(sql_disqry)
        sql_exec = cursor.fetchone()
        if len(sql_exec) > 0:
            sql_data = pd.DataFrame(cursor.fetchall())
            result = sql_data.head(n=None)
            #display(result)
      #create table in data bricks
            spark_df = spark.createDataFrame(result)
            df = spark_df.toPandas()
            #spark_df.orderBy("Emp_Id").write.mode("overwrite").saveAsTable(Source_Table_Name)
            
            pds = spark.sql("(select * FROM {})".format(Source_Table_Name))
            pdf = pds.withColumn("Hash_Key",F.sha2(F.concat('Emp_Id','Emp_name', 'Designation', 'Department', 'City', 'Salary', F.unix_timestamp('Joining_Date')),256))
            #pdf = pds.withColumn("Hash_Key",F.md5(F.concat('Emp_Id', F.trim(F.lower('Emp_name')), F.trim(F.lower('Designation')), F.trim(F.lower('Department')), F.trim(F.lower('City')), 'Salary', F.unix_timestamp('Joining_Date'))))
            pnd = pdf.toPandas()
            pd_df = pnd[['Emp_Id', 'Hash_Key']]
            #df_all = df.compare(pd_df)
            display(pd_df)
        else:
            print("No Record")
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
display_datas(sql_disqry, sql_conn)

Emp_Id,Hash_Key
4,dfe448dc6fb5bdb1aeff670d6d0bbd564418613beb725841e803eb0a5ecb2833
6,440a2ad1a4472c9703dfeac22b8a6465004710701b5ece5b25b53aa8f8db7493
7,e1999a3a4b0815464685bdcf1e80bd9635ecf8e1394b3d8d2f692cd4a9024234
8,b5cd98edfd177e0cbf476728375c679147ba61a95cd62a89e919d7cb309525c1
9,bebd1156519378d61fced177c271841cfd30d3ef4af624a81c3e37c9344f42ef
11,12b200ab4af6e9c866382a8e0f0d0770c76e06704030b1feaaa0090e5598a165
12,b6525fd429191431da1120e2e89ab4c426915fdeefacc28d0fc83472a96b7d22
13,4f16df96196a8a2eb75c579ef792b961c2c000e47070264c157a3cea7bcf86ed
14,4367e01f192da2f38fb5612007ccdefacb5477b010077542a396fc4bb0432ba8
15,aaa762d2fbeb1d33dbe7f9bee3b9ac1d809d7b323fe94a73946fc09f54277964


In [0]:
%sql

select Emp_Id, sha2(CONCAT(Emp_Id, Emp_name, Designation, Department, City, Salary),256)as Hash_Key FROM EMP_INFO

Emp_Id,Hash_Key
4,70378227284d2d1b85d67b488d51385f2cc323d63f96d4a5f99cc7ab07d67c7e
6,f08196c074c01fa415bcc7e9be254e43d29155650eed6ec480b39b260b7bbe42
7,49fe80237c92d516a2f216f8a64db2d227f46776938a727e3be96f019adaaa8b
8,49f8a852ae50dceebea5ad12d9bc25681e381fac81e7724b5c7290fe2119ef4f
9,75d5dc68724c6a0258237ea1602f639844fd38046a4b0724ca2ed737c8f315b1
11,489cecd11207885e6be97d8e14bb9fddd31f9c965859fe0cd5120462b5131430
12,2e451173cb9f2d5c9c0afef554102ceb6709dcd33d0e682eafad34694d16047e
13,7b0de2953b2f756ad8fb667dc1ebb13951b5251db1eb700f4f71aa3992924f7d
14,0832041a99419928993061a0075f8f24771e5b5cbd6a41d4b2161beffa08f3e4
15,b3492168c73df2991a39f256fb3cffa7988193c9b62bb883e9d3fc3ee84d7c7a


In [0]:
%sql
SELECT sha2('4Suresh AhirENGITSurat24100', 256)

"sha2(CAST(4Suresh AhirENGITSurat24100 AS BINARY), 256)"
70378227284d2d1b85d67b488d51385f2cc323d63f96d4a5f99cc7ab07d67c7e


In [0]:
pip install pymssql

Python interpreter will be restarted.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-ef2bfe09-7c4e-4c83-91a4-480d16a9d8bf/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.


In [0]:
str = "4Suresh AhirENGITSurat24100"
c = hashlib.sha256(str.encode())
c.hexdigest()

Out[275]: '70378227284d2d1b85d67b488d51385f2cc323d63f96d4a5f99cc7ab07d67c7e'

In [0]:
sql_disqry ="CONCAT(Emp_Id, Emp_name, Designation, Department, City, Salary)"
c = hashlib.sha256(sql_disqry.encode())
c.hexdigest()

Out[279]: '7883ceaed661ebf809bc10edfad96f9b3d6717c09240f483a7a13e4d9d6cb6a3'